# 5.Método Cadena Markov-Monte Carlo (MCMC)

In [170]:
using LinearAlgebra
using BenchmarkTools
using SparseArrays
using Random

using Plots
using PyCall
using DataFrames

## Implementación de la función que construye el sistema de ecuaciones de prueba $Ax = b$

In [171]:
function matriz_dispersa(n)
    e = ones(n)
    n2 =Int(n/2)
    diags = [-1,0,1]
    A = Matrix(spdiagm(-1 => -ones(n-1)
        ,0 => 3*ones(n),1 => -ones(n-1)))
    c = spdiagm(0 => ones(n)/2)
    ab = [x for x=1:n]
    ba = [(n+1)-x for x=1:n]
    c = Matrix(permute(c, ba, ab))

    A = A + c
    A[n2+1,n2] = -1
    A[n2,n2+1] = -1
    
    b = zeros(n,1)
    b[1] = 2.5
    b[n] = 2.5
    b[2:n-1] .= [1.5]
    b[n2:n2+1] .= [1]
    return A,b
end

matriz_dispersa (generic function with 1 method)

### Evaluación de parámetros

In [172]:

function preparametros(A,b,ϵ,δ)
    M = diagm(0 => diag(A))
    N = M-A
    T = inv(M) * N
    f = inv(M) * b
    nT, mT = size(T);

    S = fill(0, nT)
    P = fill(0., nT, mT)
    Pa = P
    [S[i] += 1 for i in 1:nT, j in 1:mT if T[i,j] != 0]
    [P[i,j]= 1/S[i] for i in 1:nT, j in 1:mT if T[i,j] != 0 ]
    Pa = [accumulate(+, P[i, 1:mT]) for i in 1:nT]
    Pi = [1/nT for i in 1:nT];
    Nc = floor((0.6745/δ)^2*((norm(f)^2)/(1-norm(T))^2)) + 1
    return Nc
end

preparametros (generic function with 1 method)

### Matriz de probabilidad acumulada pre-build con envío de parámetros, type-anotations y view

In [173]:
function mcmc_acc_par_ta(ϵ, Nc)
    Xs = fill(0., mT)
    for i in 1:mT
        W_0 = 1.0
        for s in 1:Nc
            W = W_0; point = i; X = W_0 * f[i]::Float64
            while abs(W) >= ϵ
                nextpoint  = 1::Int64
                u = rand()
                while u >= Pa[point][nextpoint]::Float64
                    nextpoint = nextpoint + 1::Int64
                end
                    W_new = W *(T[point, nextpoint]/P[point, nextpoint])::Float64
                    X = X + W_new * f[nextpoint]::Float64
                point = nextpoint::Int64
                W = W_new::Float64
            end
        Xs[i] += X::Float64
        end
    end
    Xs = Xs/Nc::Float64
end

mcmc_acc_par_ta (generic function with 2 methods)

In [174]:
function mcmc_acc_par_ta_sparse(ϵ, Nc)
    Xs = fill(0., mT)
    for i in 1:mT
        W_0 = 1.0
        for s in 1:Nc
            W = W_0; point = i; X = W_0 * f[i]::Float64
            while abs(W) >= ϵ
                nextpoint  = 1::Int64
                u = rand()
                while u >= Pa[point][nextpoint]::Float64
                    nextpoint = nextpoint + 1::Int64
                end
                    W_new =  W *(T[point, nextpoint]/P[point, nextpoint])::Float64
                    X = X + W_new * f[nextpoint]::Float64
                point = nextpoint::Int64
                W = W_new::Float64
            end
        Xs[i] += X::Float64
        end
    end
    Xs = Xs/Nc::Float64
end

mcmc_acc_par_ta_sparse (generic function with 1 method)

In [175]:
n = [6, 10, 30, 50]
A,b = matriz_dispersa(n[1])
A_sparse = sparse(A)
b_sparse = sparse(b)
ϵ = 0.1
δ = 0.1 

0.1

In [176]:
Nc = preparametros(A,b,ϵ,δ)
Nc_sparse = preparametros(A_sparse,b_sparse,ϵ,δ)

8623.0

In [177]:
@benchmark mcmc_acc_par_ta($ϵ, $Nc)

BenchmarkTools.Trial: 69 samples with 1 evaluation.
 Range (min … max):  69.467 ms … 80.641 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     71.937 ms              ┊ GC (median):    2.90%
 Time  (mean ± σ):   73.219 ms ±  2.515 ms  ┊ GC (mean ± σ):  2.61% ± 0.89%

            ▂█▄▇                                               
  ▆▁▁▁▁▁▁▁▁▁████▆▆▁▁▃▃▃▁▆▁▅▃▁▁▃▁▃▁▅▃▃▆▁▃▁▁▁▁▃▆▃▁▁▁▃▁▁▁▁▁▁▁▁▃▃ ▁
  69.5 ms         Histogram: frequency by time        79.9 ms <

 Memory estimate: 39.28 MiB, allocs estimate: 2574415.

In [178]:
@benchmark mcmc_acc_par_ta_view($ϵ, $Nc_sparse)

BenchmarkTools.Trial: 63 samples with 1 evaluation.
 Range (min … max):  72.529 ms … 95.101 ms  ┊ GC (min … max): 0.00% … 2.50%
 Time  (median):     75.156 ms              ┊ GC (median):    2.74%
 Time  (mean ± σ):   80.202 ms ±  6.806 ms  ┊ GC (mean ± σ):  2.47% ± 0.93%

       ▁█                                                      
  ▅▁▁▁▁██▆▃▁▃▁▃▃▁▃▁▁▃▁▁▁▁▁▁▁▁▁▁▁▁▁▃▁▁▁▁▃▁▃▁▄▄▄▃▃▃▃▄▁▁▅▁▄▃▃▃▁▄ ▁
  72.5 ms         Histogram: frequency by time        91.3 ms <

 Memory estimate: 39.28 MiB, allocs estimate: 2573938.

In [182]:
X  = @btime mcmc_acc_par_ta($ϵ, $Nc);
Xs = @btime mcmc_acc_par_ta_view($ϵ, $Nc_sparse);

  70.164 ms (2573839 allocations: 39.27 MiB)
  73.933 ms (2573086 allocations: 39.26 MiB)


In [183]:
norm(b-A*X)

0.025381644133328815

In [184]:
norm(b-A*Xs)

0.02811680175380445